In [1]:
import sys
# sys.path.append("/home/riccardo/mmdetection3d")
# sys.path.append("/root/workspace/mmdeploy")
from mmdet3d.apis import LidarDet3DInferencer
from mmdeploy.apis.inference import inference_model
from mmdeploy.apis.inference import get_model
import os
import torch
import os
import numpy as np
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
import onnxruntime as ort
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

In [2]:
# initialize inferencer 
# execute >> pip install -v -e . in mmdetection3d folder 
# CHANGE THE center_mode in loca_visualizer accordingly (KITTI -> center_mode = 'lidar_bottom', CUSTOM -> center_mode = whatever)

# inferencer = LidarDet3DInferencer('pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car')
# inferencer = LidarDet3DInferencer('pointpillars_donaset_container-car')
# inferencer = LidarDet3DInferencer('pointpillars_kittilidar-car') 
# inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car",scope="mmdet3d")
# inferencer = LidarDet3DInferencer("pointpillars_lidar_vegas_iris-car_360",scope="mmdet3d")
inferencer = LidarDet3DInferencer("pointpillars_hydris_car",scope="mmdet3d")


Loads checkpoint by local backend from path: /home/riccardo/LidarObjDetection2/mmdetection3d/work_dirs/pointpillars_hydris_car/iter_6500.pth
06/11 10:53:46 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


/root/workspace/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [12]:
inferencer.model.data_preprocessor

Det3DDataPreprocessor(
  (voxel_layer): VoxelizationByGridShape(voxel_size=[0.16, 0.16, 6], grid_shape=[872, 248, 1], point_cloud_range=[-60, -19.84, -3, 79.52, 19.84, 3], max_num_points=32, max_voxels=(16000, 40000), deterministic=False)
)

# Prediction Tests

In [3]:
dataset_dir = "../data/Hydris/"

### Create Validation

In [5]:
import os

def copy_bin_files(source_txt, bin_folder_path,destination_folder):


  # Check if source text file exists
  if not os.path.exists(source_txt):
    print(f"Error: Source text file '{source_txt}' does not exist.")
    return

  # Check if destination folder exists, create it if not
  if not os.path.exists(destination_folder):
    try:
      os.makedirs(destination_folder)
      print(f"Destination folder '{destination_folder}' created.")
    except OSError as e:
      print(f"Error creating destination folder: {e}")
      return

  # Open the source text file
  try:
    with open(source_txt, 'r') as file:
      lines = file.readlines()
  except OSError as e:
    print(f"Error opening source text file: {e}")
    return

  # Process each line in the text file
  for line in lines:
    # Remove leading/trailing whitespace and newline character
    filename = line.strip() +".bin"

    # Check if the filename ends with .bin
    if not filename.endswith('.bin'):
      print(f"Warning: Skipping non-binary file '{filename}'.")
      continue

    # Construct source and destination paths
    source_path = os.path.join(bin_folder_path, filename)
    dest_path = os.path.join(destination_folder, filename)

    # Check if source file exists
    if not os.path.exists(source_path):
      print(f"Warning: Source file '{source_path}' does not exist. Skipping...")
      continue

    # Copy the file
    try:
      with open(source_path, 'rb') as source, open(dest_path, 'wb') as destination:
        for chunk in iter(lambda: source.read(1024), b''):
          destination.write(chunk)
      print(f"Copied '{filename}' to '{destination_folder}'.")
    except OSError as e:
      print(f"Error copying file '{filename}': {e}")

# Example usage
source_txt_path = dataset_dir +"/ImageSets/val.txt"  # Replace with your actual path
destination_folder_path = dataset_dir +"/validation"  # Replace with your actual path
bin_folder_path = dataset_dir +"/training/velodyne"
copy_bin_files(source_txt_path, bin_folder_path, destination_folder_path)


Error copying file '000043.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000043.bin'
Error copying file '000295.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000295.bin'
Error copying file '000139.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000139.bin'
Error copying file '000150.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000150.bin'
Error copying file '000255.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000255.bin'
Error copying file '000079.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000079.bin'
Error copying file '000431.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000431.bin'
Error copying file '000126.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000126.bin'
Error copying file '000056.bin': [Errno 13] Permission denied: '../data/Hydris//validation/000056.bin'
Error copying file '000355.bin': [Errno 13] Permission denied: '../data/H

### Inference

In [4]:
# inference
# pcl = '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'
pcl = os.path.join(dataset_dir,"validation")
# pcl = '../data/DonaSet/training/velodyne/'
# pcl = './data/falcon/falcon1.bin'
inputs = dict(points=pcl)
inf_res = inferencer(inputs, show=True,pred_score_thr=0.3, batch_size=1)
inf_res

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraPar

SystemExit: 0

/home/riccardo/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [29]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3,batch_size=1)
end = time.time()
print("Time Taken {} ms".format((end-start)*1000))

Time Taken 112.63775825500488 ms


In [ ]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

def prepare_input(inputs,batch_size=1):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [ ]:
prep_input = prepare_input({"points": pcl}, batch_size=1)

In [ ]:
model = inferencer.model


In [ ]:
import time 
import torch

torch.manual_seed(42)
s = time.time()

prep_input = prepare_input({"points": pcl}, batch_size=1) # Preprocess
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor") # Forward
preds  = model.bbox_head.predict_by_feat(*res) # Postprocess

print("Time: {} ms".format((time.time()-s)*1000))
print(preds)

In [ ]:
inf_res = inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)
inf_res

Convert Manually the tensor to predictions

In [ ]:
from mmdet3d.models.dense_heads.anchor3d_head import Anchor3DHead

# bbox_head=dict(
#     type='Anchor3DHead',
#     num_classes=3,
#     in_channels=384,
#     feat_channels=384,
#     use_direction_classifier=True,
#     assign_per_class=True,
#     anchor_generator=dict(
#         type='AlignedAnchor3DRangeGenerator',
#         ranges=[
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -1.78, 69.12, 39.68, -1.78],
#         ],
#         sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
#         rotations=[0, 1.57],
#         reshape_out=False),
#     diff_rad_by_sin=True,
#     bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder')
# )
# model.bbox_head
head = Anchor3DHead(
    num_classes=1,
    in_channels=384,
    feat_channels=384,
    use_direction_classifier=True,
    assign_per_class=True,
    anchor_generator=dict(
                type='AlignedAnchor3DRangeGenerator',
                ranges=[[0, -39.68, -1.78, 69.12, 39.68, -1.78]],
                sizes=[[3.9, 1.6, 1.56]],
                rotations=[0, 1.57],
                reshape_out=True),
    diff_rad_by_sin=True,
    bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
    test_cfg=model.test_cfg
    )
# preds  = head.predict_by_feat(*res)
preds  = model.bbox_head.predict_by_feat(*res)
preds

In [ ]:
preds[0]["bboxes_3d"],pred_res[0]["bboxes_3d"],inf_res["predictions"][0]["bboxes_3d"]

# Model Analysis

In [13]:
model = inferencer.model
pcl_path = "../data/Dataset_Lidar_20240111_vegas_ontrack_01/training/velodyne/000050.bin"
model_path = "deployed_models/iris_360_2/end2end.onnx"

In [58]:
def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False, deterministic=True):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[-60, -19.84, -3, 79.52, 19.84, 3],
                # point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 3],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000),
                deterministic=deterministic))
        if not deterministic:
            preprocessor.cuda()
        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

## Inference time comparison

### Pytorch CPU

In [16]:
s0 = time.time()
model.cpu()
inputs = prepare_input({"points": pcl_path}, batch_size=1, device = "cpu")["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Pre: 4.081964492797852 ms
Inf: 260.21838188171387 ms
Post: 2.9685497283935547 ms
Total: 263.20791244506836 ms


### Pytorch GPU

In [14]:
s0 = time.time()
model.cuda()
inputs = prepare_input({"points": pcl_path}, batch_size=1,deterministic=True)["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Pre: 4.9419403076171875 ms
Inf: 16.715526580810547 ms
Post: 33.89549255371094 ms
Total: 50.63295364379883 ms


### TensorRT Provider (ONNX Runtime)

In [20]:
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

2024-06-10 12:46:40.493606214 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:40 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-06-10 12:46:40.493627167 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:40 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-06-10 12:46:40.598072051 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:40 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-06-10 12:46:40.598253319 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:40 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


### TensorRT Provider (ONNX Runtime) - FP16

In [ ]:
!pip install onnxmltools

In [ ]:
from onnx import load
from onnxmltools.utils.float16_converter import convert_float_to_float16
from onnx import save
onnx_model = load(model_path)
new_onnx_model = convert_float_to_float16(onnx_model)
save(new_onnx_model,"end2end_fp16.onnx")

In [16]:
ort_sess = ort.InferenceSession("deployed_models/iris_front/end2end_fp16.onnx",providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

2024-06-10 12:45:57.429920415 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:45:57 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-06-10 12:45:57.429948420 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:45:57 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-06-10 12:46:01.681154268 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:01 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-06-10 12:46:01.681176955 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-06-10 10:46:01 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [19]:
s0= time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,device="cuda")["inputs"]

onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy().astype(np.float16), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
outputs = [arr.astype(np.float32) for arr in outputs]
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

Pre: 45.86482048034668 ms
Inf: 37.64224052429199 ms
Post: 33.80393981933594 ms
Total: 117.63334274291992 ms


### CUDA Provider  (ONNX Runtime)

In [12]:
ort_sess = ort.InferenceSession(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [15]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Inf: 68.58515739440918 ms
Post: 15.950441360473633 ms
Total: 84.56301689147949 ms


### CPU Provider  (ONNX Runtime)

In [ ]:
ort_sess = ort.InferenceSession(model_path,providers=[ 'CPUExecutionProvider'])

In [ ]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

###  TensorRT

In [59]:


model_cfg = "../configs/pointpillars/pointpillars_hydris_car.py"
deploy_cfg = "../../mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4_fp16.py"
backend_files = "deployed_models/hydris/end2end_fp16.engine"
img = "../data/Hydris/training/velodyne/000001.bin"
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

06/11 11:30:01 - mmengine - INFO - Successfully loaded tensorrt plugins from /root/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_tensorrt_ops.so
06/11 11:30:01 - mmengine - INFO - Successfully loaded tensorrt plugins from /root/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_tensorrt_ops.so


In [60]:
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

In [63]:
p = np.zeros((1000,4))
torch_tensor = torch.tensor(p)
a = {} 
a["inputs"] = {}
a["inputs"]["points"] = [torch_tensor]
b = tensorrt_model.data_preprocessor(a)
b["inputs"]["voxels"]["voxels"] = b["inputs"]["voxels"]["voxels"].cuda()

In [77]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
s2 =time.time()
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

Pre: 9.975194931030273 ms
Inf: 1.264333724975586 ms
Post: 15.738487243652344 ms
Total: 26.99875831604004 ms


In [88]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
s2 =time.time()
preds  = bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

Pre: 13.08298110961914 ms
Inf: 0.8113384246826172 ms
Post: 15.533208847045898 ms
Total: 29.454708099365234 ms


In [87]:
from mmengine.config import Config
from mmdet3d.models.dense_heads import Anchor3DHead

model_cfg = "/home/riccardo/LidarObjDetection2/mmdetection3d/configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car.py"
cfg = Config.fromfile(model_cfg)

cfg["model"]["bbox_head"].pop("type")
bbox_head = Anchor3DHead(**cfg["model"]["bbox_head"])
bbox_head.test_cfg = cfg["model"]["test_cfg"]

/root/workspace/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


### TensorRT Fp16

In [28]:


model_cfg = "../configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car.py"
deploy_cfg = "../../mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4_fp16.py"
backend_files = "deployed_models/iris_360_2/end2end_fp16.engine"
img = "../data/Dataset_Lidar_20240111_vegas_ontrack_01/training/velodyne/000001.bin"
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

06/10 12:34:24 - mmengine - INFO - Successfully loaded tensorrt plugins from /root/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_tensorrt_ops.so
06/10 12:34:24 - mmengine - INFO - Successfully loaded tensorrt plugins from /root/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_tensorrt_ops.so


In [36]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
s2 =time.time()
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Pre: {} ms".format((s1-s0)*1000))
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s0)*1000))

Pre: 16.86406135559082 ms
Inf: 5.480051040649414 ms
Post: 28.685331344604492 ms
Total: 51.058053970336914 ms


### Visualize

In [18]:
from mmdet3d.structures import Det3DDataSample
ds = Det3DDataSample()
ds.pred_instances_3d = preds[0]

inferencer.visualize([{"points": pcl_path}], [ds],show=True,pred_score_thr=0.3)

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


SystemExit: 0

## Voxelization algorithm comparison

In [39]:
from mmdet3d.models.data_preprocessors import Det3DDataPreprocessor


def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False, deterministic=True):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[-60, -19.84, -3, 79.52, 19.84, 3],
                # point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 3],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000),
                deterministic=deterministic))
        if not deterministic:
            preprocessor.cuda()
        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            # print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [59]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False, deterministic=False, device="cuda")["inputs"]
print("Non deterministic voxel -> " +str((time.time()-s0)*1000))

s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False, deterministic=True, device="cuda")["inputs"]
print("Deterministic voxel -> " +str((time.time()-s0)*1000))


Non deterministic voxel -> 3.5941600799560547
Deterministic voxel -> 4.848241806030273
